In [1]:
# Importamos las bibliotecas necesarias
import pandas as pd
import ast

# Lectura y carga del archivo JSON:
# Abrimos el archivo JSON y lo leemos línea por línea. Utilizamos ast.literal_eval para convertir cada línea en un diccionario y lo agregamos a la lista "rows".
rows = []

with open("data/australian_user_reviews.json", encoding='MacRoman') as file:
    for line in file.readlines():
        rows.append(ast.literal_eval(line))

# Creamos un DataFrame llamado "df_custom_reviews" a partir de la lista "rows".
# Este DataFrame contiene los datos anidados del archivo JSON.
df_custom_reviews = pd.DataFrame(rows)

# Visualizamos la estructura del conjunto de datos de revisiones
df_custom_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [2]:
# Importamos la librería TextBlob, que nos permitirá realizar un análisis de sentimiento sobre las reseñas del DataFrame.

from textblob import TextBlob

# Definimos la función `sentiment_analysis` para evaluar el sentimiento de las reseñas.
# Esta función toma una lista que contiene un diccionario con información de la reseña.

def sentiment_analysis(review):
    """
    Analiza el sentimiento de una reseña y asigna un valor numérico.

    Parameters:
    - review (list): Lista que contiene un diccionario con información de la reseña.

    Returns:
    - int: 0 para sentimiento negativo, 1 para neutral, 2 para positivo.
    """
    # Verificamos si la entrada es una lista y si tiene al menos un elemento.
    if isinstance(review, list) and len(review) > 0:
        # Obtenemos el texto de la reseña del diccionario.
        texto = review[0].get('review', '')
        
        # Calculamos la polaridad del sentimiento utilizando TextBlob.
        sentiment = TextBlob(texto).sentiment.polarity

        # Asignamos un valor numérico en función de la polaridad calculada.
        if sentiment < -0.2:
            return 0  # Sentimiento negativo (Malo)
        elif -0.2 <= sentiment <= 0.2:
            return 1  # Sentimiento neutral
        else:
            return 2  # Sentimiento positivo
    else:
        # Valor predeterminado para reseñas faltantes o mal formateadas.
        return 1  # Neutral

# Aplicamos la función `sentiment_analysis` a la columna 'reviews' y creamos la nueva columna 'sentiment_analysis'.
df_custom_reviews['sentiment_analysis'] = df_custom_reviews['reviews'].apply(sentiment_analysis)

# Mostramos el DataFrame resultante que incluye la nueva columna 'sentiment_analysis'.
df_custom_reviews

,user_id,user_url,reviews,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",1
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",1
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",2
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",2
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",1
...,...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la...",2
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l...",1
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',...",1
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l...",2


In [3]:
# La lista data_desanidades se utiliza para almacenar datos desanidados de reseñas y otros detalles.
data_desanidada = []

# Iteramos sobre cada fila del DataFrame df_custom_reviews.
for index, row in df_custom_reviews.iterrows():
    # Extraemos valores importantes de la fila actual.
    user_id = row['user_id']
    user_url = row['user_url']
    sentiment_analysis_value = row['sentiment_analysis']  # Obtener el valor del análisis de sentimiento
    reviews = row['reviews']

    # Iteramos sobre cada reseña en la lista de reseñas para la fila actual.
    for i in reviews:
        # Creamos un nuevo diccionario que almacena detalles relevantes de la fila actual.
        new_row = {
            'user_id': user_id,
            'user_url': user_url,
            'reviews': reviews,
            'sentiment_analysis': sentiment_analysis_value,
            'funny': i.get('funny', ''),  # Detalles humorísticos
            'posted': i.get('posted', ''),  # Fecha de publicación
            'last_edited': i.get('last_edited', ''),  # Última edición
            'item_id': i.get('item_id', ''),  # ID del artículo
            'helpful': i.get('helpful', ''),  # Valor de utilidad
            'recommend': i.get('recommend', False),  # Recomendación
            'review': i.get('review', '')  # Texto de la reseña
        }
        # Agregamos el nuevo diccionario a la lista data_desanidada.
        data_desanidada.append(new_row)

# Creamos un nuevo DataFrame df_reviews_desanidada a partir de la lista data_desanidada.
df_reviews_desanidada = pd.DataFrame(data_desanidada)

# Eliminamos columnas innecesarias para un análisis más limpio.
df_reviews_desanidada.drop(['reviews', 'helpful', 'review', 'last_edited', 'funny'], axis=1, inplace=True)

df_reviews_desanidada

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted November 5, 2011.",1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted July 15, 2011.",22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted April 21, 2011.",43110,True
3,js41637,http://steamcommunity.com/id/js41637,1,"Posted June 24, 2014.",251610,True
4,js41637,http://steamcommunity.com/id/js41637,1,"Posted September 8, 2013.",227300,True
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,Posted July 10.,70,True
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,Posted July 8.,362890,True
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,Posted July 3.,273110,True
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,Posted July 20.,730,True


In [4]:
# Importamos el módulo parser de la librería dateutil
from dateutil import parser

# Definimos una función para analizar la fecha y manejar posibles errores
def parse_date(date_str):
    try:
        # Intentamos analizar la fecha, eliminando la parte "Posted " y usando el modo 'fuzzy' para manejar formatos diversos
        return parser.parse(date_str.replace("Posted ", ""), fuzzy=True)
    except ValueError:
        # En caso de error, devolvemos None
        return None

# Aplicamos la función de análisis de fecha a la columna 'posted' y reemplazamos filas con fechas incorrectas con NaN
df_reviews_desanidada['posted'] = df_reviews_desanidada['posted'].apply(parse_date)

# Eliminamos filas con fechas incorrectas (NaN en la columna 'posted')
df_reviews_desanidada = df_reviews_desanidada.dropna(subset=['posted'])

# Mostramos las reseñas con las fechas correctas
df_reviews_desanidada

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-11-05,1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-07-15,22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-04-21,43110,True
3,js41637,http://steamcommunity.com/id/js41637,1,2014-06-24,251610,True
4,js41637,http://steamcommunity.com/id/js41637,1,2013-09-08,227300,True
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,2024-07-10,70,True
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,2024-07-08,362890,True
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,2024-07-03,273110,True
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,2024-07-20,730,True


In [6]:
# Se crea una lista vacía llamada rows1. Esta lista se utilizará para almacenar los datos leídos del archivo JSON.
rows1 = []  

# Abre el archivo JSON llamado "australian_users_items.json"
with open("data/australian_users_items.json", encoding='MacRoman') as f:
    
    # Se itera a través de cada línea del archivo y se devuelve una lista de cadenas (una cadena por línea)
    for line in f.readlines():
     
        # Cada línea se evalúa utilizando la función ast.literal_eval()
        rows1.append(ast.literal_eval(line))

# Carga los datos en un DataFrame.
df_custom_items = pd.DataFrame(rows1)

# Muestra los datos del DataFrame
df_custom_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [7]:

# Creamos una lista para almacenar los datos desanidados
data_desanidada = []

# Iteramos sobre cada fila del DataFrame original (df_custom_items)
for index, row in df_custom_items.iterrows():
    # Extraemos información relevante de la fila
    user_id = row['user_id']
    steam_id = row['steam_id']
    user_url = row['user_url']
    
    # Iteramos sobre cada elemento en la lista 'items' de la fila actual
    for item in row['items']:
        # Creamos un nuevo diccionario con los valores desanidados
        new_row = {
            'user_id': user_id,
            'steam_id': steam_id,
            'user_url': user_url,
            'item_id': item['item_id'],
            'item_name': item['item_name'],
            'playtime_forever': item['playtime_forever'],
            'playtime_2weeks': item['playtime_2weeks']
        }
        
        # Agregamos el diccionario a la lista de datos desanidados
        data_desanidada.append(new_row)

# Creamos un nuevo DataFrame (df_custom_items_desanidado) a partir de la lista desanidada
df_custom_items_desanidado = pd.DataFrame(data_desanidada)

# El DataFrame modificado (df_custom_items_desanidado) ahora contiene datos desanidados de 'items'
df_custom_items_desanidado



,user_id,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...,...
5153204,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,346330,BrainBread 2,0,0
5153205,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0,0
5153206,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4,4


In [8]:
# Lee el archivo output_steam_games y carga los datos en un DataFrame
df_steam_games_original = pd.read_json("data/output_steam_games.json", lines=True)

# Elimina registros con valores NaN en todos los campos
df_steam_games_original.dropna(how='all', inplace=True)

# Muestra el DataFrame con los valores eliminados
df_steam_games_original

FileNotFoundError: File json/output_steam_games.json does not exist

In [ ]:
# Convierte en 0 los registros que no poseen ID
df_steam_games_original['id'] = df_steam_games_original['id'].fillna(0).astype(int)

# Convierte en enteros los ID del DataFrame df_steam_games_original
df_steam_games_original['id'] = df_steam_games_original['id'].astype(int)

# Muestra el DataFrame df_steam_games_original después de las transformaciones
df_steam_games_original

In [ ]:
# Guardo los DataFrames modificados en formato CSV y Parquet.

# Guardar el DataFrame df_steam_games_original en un archivo CSV sin incluir el índice
df_steam_games_original.to_csv('./data/steam_games.csv', index=False)

# Guardar el DataFrame df_reviews_desanidada en un archivo CSV sin incluir el índice
df_reviews_desanidada.to_csv('./data/reviews.csv', index=False)

# Guardar el DataFrame df_custom_items_desanidado en formato Parquet sin incluir el índice
df_custom_items_desanidado.to_parquet('./data/items.parquet', index=False)

# Leer los archivos guardados y cargarlos en nuevos DataFrames

# Leer el archivo CSV y cargarlo en el DataFrame df_steam
df_steam = pd.read_csv('./data/steam_games.csv')

# Leer el archivo Parquet y cargarlo en el DataFrame df_items
df_items = pd.read_parquet('./data/items.parquet')

# Leer el archivo CSV y cargarlo en el DataFrame df_reviews
df_reviews = pd.read_csv('./data/reviews.csv')

In [ ]:
# Mostrar archivo CSV
df_reviews

In [ ]:
# Mostrar archivo CSV
df_items

In [ ]:
# Mostrar archivo CSV
df_steam

In [ ]:

# Averiguamos los 3 juegos más recomendados.

# Filtrar solo las filas con recommend=True
df_reviews_true = df_reviews[df_reviews['recommend']]

# Contar la cantidad de recomendaciones por item_id
recomendaciones_por_item = df_reviews_true.groupby('item_id')['recommend'].count().reset_index()

# Ordenar en orden descendente según la cantidad de recomendaciones
recomendaciones_por_item = recomendaciones_por_item.sort_values(by='recommend', ascending=False)

# Tomar los tres primeros ítems más recomendados
top_3_recomendados = recomendaciones_por_item.head(3)

# Combinar con df_steam para obtener los nombres de los ítems
top_3_mas_recomendados = pd.merge(top_3_recomendados, df_steam[['id', 'app_name']], left_on='item_id', right_on='id', how='left')

# Imprimir el resultado
print(top_3_mas_recomendados)

In [ ]:
import matplotlib.pyplot as plt  # Importa la biblioteca Matplotlib para visualización de datos.

# Informaremos los juegos con mayor tiempo jugado
top_juegos_jugados = df_items.sort_values(by='playtime_forever', ascending=False).head(10)

plt.figure(figsize=(6, 2))  # Ajusta el tamaño del gráfico.

# Graficar los datos en un gráfico de barras horizontales
plt.barh(top_juegos_jugados['item_name'], top_juegos_jugados['playtime_forever'], color='red')

plt.xlabel('Tiempo total jugado')  # Etiqueta del eje X
plt.ylabel('Juegos')                # Etiqueta del eje Y
plt.title('Top Juegos con Mayor Tiempo Jugado')  # Título del gráfico

plt.gca().invert_yaxis()  # Invertir el eje Y para que el juego con más tiempo jugado esté en la parte superior.

# Mostrar el gráfico
plt.show()

In [ ]:
# Análisis de la cantidad de juegos lanzados por mes

# Copiamos el DataFrame original df_steam para evitar modificar el conjunto de datos original
df_steam_copia = df_steam.copy()

# Convertimos la columna "release_date" al formato de fecha y eliminamos las filas con valores incorrectos (NaN)
df_steam_copia['release_date'] = pd.to_datetime(df_steam_copia['release_date'], errors='coerce')

# Creamos una nueva columna "release_month" que contiene el mes de lanzamiento
df_steam_copia['release_month'] = df_steam_copia['release_date'].dt.to_period('M')

# Calculamos la cantidad de juegos lanzados por mes
cantidad_juegos_por_mes = df_steam_copia.groupby('release_month').size().reset_index(name='cantidad_juegos')

# El DataFrame "cantidad_juegos_por_mes" ahora contiene la cantidad de juegos lanzados por mes
print(cantidad_juegos_por_mes)

In [ ]:
# Visualización de la cantidad de juegos lanzados en los últimos 12 meses

# Selecciona los últimos 12 meses del DataFrame cantidad_juegos_por_mes
ultimos_12_meses = cantidad_juegos_por_mes[-12:]

# Crea un gráfico de barras apiladas con tamaño personalizado
plt.figure(figsize=(10, 6))

# Extrae los meses y la cantidad de juegos, convirtiendo los meses a formato numérico
meses = ultimos_12_meses['release_month'].dt.month.astype(int)
cantidad_juegos = ultimos_12_meses['cantidad_juegos']

# Crea el gráfico de barras
plt.bar(meses, cantidad_juegos)

# Configura el eje x para que muestre los meses de forma legible
plt.xlabel('Mes de Lanzamiento')
plt.ylabel('Cantidad de Juegos Lanzados')
plt.title('Cantidad de Juegos Lanzados por Mes (Últimos 12 Meses)')

# Ajusta el diseño del gráfico para evitar cortar las etiquetas
plt.tight_layout()

# Muestra el gráfico
plt.show()

In [ ]:
# Recomendación de juegos basada en contenido

# Especifica el tamaño de la muestra
tamano_muestra = 100  

# Toma una muestra aleatoria del DataFrame original
muestra_aleatoria = df_steam.sample(n=tamano_muestra, random_state=42)  

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Obtén el juego con el item_id proporcionado por el usuario
user_item_id = 10  # Reemplaza con el item_id proporcionado por el usuario
target_game = muestra_aleatoria[muestra_aleatoria['id'] == user_item_id]

# Combina las etiquetas (tags) y géneros en una sola cadena de texto
# Limpia los valores nulos en las columnas 'tags' y 'genres' y convierte todo a cadenas de texto
target_game_tags_and_genres = ' '.join(target_game['tags'].fillna('').astype(str) + ' ' + target_game['genres'].fillna('').astype(str))

# Crea un vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Aplica el vectorizador al juego de referencia y a todos los juegos
# Limpia los valores nulos en las columnas 'tags' y 'genres' y convierte todo a cadenas de texto
muestra_aleatoria['tags_and_genres'] = muestra_aleatoria['tags'].fillna('').astype(str) + ' ' + muestra_aleatoria['genres'].fillna('').astype(str)
tfidf_matrix = tfidf_vectorizer.fit_transform([target_game_tags_and_genres] + muestra_aleatoria['tags_and_genres'].tolist())

# Calcula la similitud entre el juego de referencia y todos los demás juegos
similarity_matrix = cosine_similarity(tfidf_matrix)

# Ordena los juegos por similitud y obtén los índices de los juegos más similares
similar_games_indices = similarity_matrix[0].argsort()[::-1]

# Recomienda los juegos más similares (puedes ajustar el número de recomendaciones)
num_recommendations = 5
recommended_games = muestra_aleatoria.reset_index(drop=True).loc[similar_games_indices[1:num_recommendations + 1]]

# Devuelve la lista de juegos recomendados
recommended_games

In [ ]:
# Implementación de Recomendación de Juegos Basada en Contenido con FastAPI

# Importaciones necesarias
from fastapi import FastAPI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Inicialización de la aplicación FastAPI
app2 = FastAPI()

# Cargar el DataFrame df_steam desde tu archivo CSV
df_steam = pd.read_csv('./Data/steam_games.csv')

# Definición de la ruta para la recomendación de juegos basada en contenido
@app2.get("/recomendacion_juego/{product_id}")
async def recomendacion_juego(product_id: int):
    try:
        # Obtén el juego de referencia
        target_game = df_steam[df_steam['id'] == product_id]

        if target_game.empty:
            return {"message": "No se encontró el juego de referencia."}

        # Combina las etiquetas (tags) y géneros en una sola cadena de texto
        target_game_tags_and_genres = ' '.join(target_game['tags'].fillna('').astype(str) + ' ' + target_game['genres'].fillna('').astype(str))

        # Crea un vectorizador TF-IDF
        tfidf_vectorizer = TfidfVectorizer()

        # Configura el tamaño del lote para la lectura de juegos
        chunk_size = 100  # Tamaño del lote 
        similarity_scores = None

        # Procesa los juegos por lotes utilizando chunks
        for chunk in pd.read_csv('./Data/steam_games.csv', chunksize=chunk_size):
            # Combina las etiquetas (tags) y géneros de los juegos en una sola cadena de texto
            chunk_tags_and_genres = ' '.join(chunk['tags'].fillna('').astype(str) + ' ' + chunk['genres'].fillna('').astype(str))

            # Aplica el vectorizador TF-IDF al lote actual de juegos y al juego de referencia
            tfidf_matrix = tfidf_vectorizer.fit_transform([target_game_tags_and_genres, chunk_tags_and_genres])

            # Calcula la similitud entre el juego de referencia y los juegos del lote actual
            if similarity_scores is None:
                similarity_scores = cosine_similarity(tfidf_matrix)
            else:
                similarity_scores = cosine_similarity(tfidf_matrix, X=similarity_scores)

        if similarity_scores is not None:
            # Obtiene los índices de los juegos más similares
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            # Recomienda los juegos más similares (puedes ajustar el número de recomendaciones)
            num_recommendations = 5
            recommended_games = df_steam.loc[similar_games_indices[1:num_recommendations + 1]]

            # Devuelve la lista de juegos recomendados con información específica
            return recommended_games[['app_name', 'tags', 'genres']].to_dict(orient='records')

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

In [ ]:

def userdata(user_id):
    """
    Calcula información relevante sobre un usuario específico.

    Parameters:
    - user_id (int): Identificación única del usuario.

    Returns:
    - dict: Diccionario que contiene información sobre el usuario, incluyendo el dinero gastado,
            el porcentaje de recomendación promedio y la cantidad de juegos poseídos.
    """

    # Paso 1: Filtrar las revisiones del usuario específico
    user_reviews = df_reviews[df_reviews['user_id'] == user_id]
    
    # Paso 2: Filtrar los juegos jugados por el usuario
    game_ids = user_reviews['item_id'].unique()
    user_steam_games = df_steam[df_steam['id'].astype(int).isin(game_ids)]
    
    # Paso 3: Calcular la cantidad de dinero gastado por el usuario
    user_steam_games.loc[:, 'price'] = user_steam_games['price'].str.replace('$', '').astype(float)
    money_spent = user_steam_games['price'].sum()
    
    # Paso 4: Calcular el porcentaje de recomendación promedio de los juegos jugados por el usuario
    user_reviews.loc[:, 'recommend'] = user_reviews['recommend'].astype(bool)
    recommend_percentage = user_reviews['recommend'].mean() * 100
    
    # Paso 5: Calcular la cantidad de items que posee el usuario
    num_items = len(game_ids)
    
    # Paso 6: Crear un diccionario con los resultados
    user_data = {
        "money_spent": money_spent,
        "recommend_percentage": recommend_percentage,
        "num_items": num_items
    }
    
    # Paso 7: Devolver el diccionario con la información del usuario
    return user_data

In [ ]:
# Reemplazar "tu_user_id_aqui" con el ID del usuario que deseas consultar
user_id = "76561197970982479"

# Llama a la función userdata con el user_id específico
resultado_usuario = userdata(user_id)

# Imprime los resultados
print("Cantidad de dinero gastado:", resultado_usuario["money_spent"])
print("Porcentaje de recomendación:", resultado_usuario["recommend_percentage"])
print("Cantidad de items:", resultado_usuario["num_items"])

In [ ]:
from fastapi import FastAPI
import pandas as pd

app = FastAPI()

# Define un endpoint para obtener datos de un usuario según su user_id
@app.get("/userdata/{user_id}")
async def userdata(user_id: str):
    try:
        # Inicializa variables
        total_money_spent = 0
        recommend_count = 0
        total_reviews = 0
        unique_item_ids = set()

        # Tamaño del lote para la lectura de reviews
        chunk_size = 100000
        user_reviews_generator = pd.read_csv('./Data/reviews.csv', chunksize=chunk_size)

        # Itera a través de los lotes de reviews
        for chunk in user_reviews_generator:
            user_reviews = chunk[chunk['user_id'] == user_id]

            # Procesa los datos del lote actual
            total_money_spent += user_reviews.merge(df_steam[['id', 'price']], left_on='item_id', right_on='id', how='inner')['price'].sum()
            recommend_count += user_reviews['recommend'].sum()
            total_reviews += len(user_reviews)
            unique_item_ids.update(user_reviews['item_id'].unique())

        # Calcula el porcentaje de recomendaciones
        recommend_percentage = (recommend_count / total_reviews) * 100 if total_reviews > 0 else 0

        # Obtiene la cantidad de items únicos
        num_unique_items = len(unique_item_ids)

        # Crea un diccionario con los resultados
        user_data = {
            "Total dinero gastado": total_money_spent,
            "Porcentaje de recomendación": recommend_percentage,
            "Cantidad de items únicos": num_unique_items
        }

        return user_data

    except Exception as e:
        # Manejo de errores
        return {"message": f"Error: {str(e)}"}
    return user_data
